In [ ]:
""" Задачи """

Задача Численный эксперимент

В наш А/В-тест попадают все пользователи, совершавшие покупки до 28 марта.

 
Целевая метрика — средняя выручка с клиента за время эксперимента. Целевую метрику считаем на неделе с 21 по 28 марта. Уровень значимости — 0.05. Критерий — тест Стьюдента. Размер групп — 1000. Ожидаемый эффект — средняя выручка увеличится на 10%.

Нужно оценить вероятности ошибок II рода для трёх вариантов добавления эффекта:

1. Добавление константы ко всем значениям;

2. Умножение на константу всех значений;

3. Добавление константы к 2.5% значений.

Для решения используйте данные из файла 2022-04-01T12_df_sales.csv.


In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from scipy import stats
from tqdm.notebook import tqdm

In [2]:
def read_file(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'Файл {file_name} не найден')
    return pd.read_csv(file_name)

In [3]:
df = read_file('df_sales.csv')

In [4]:
start_date = datetime(2022, 3, 21)
end_date = datetime(2022, 3, 28)

df['date'] = pd.to_datetime(df['date'])
#Определим список всех пользователей, которые делали покупки до конца периода (до 28 марта)
df_users = df[df['date'] < end_date][['user_id']].drop_duplicates()

#Для каждого из этих пользователей считает сумму их покупок за период с 21 по 27 марта.
df_metrics = df[(df['date']>=start_date)
        & (df['date']<end_date)].groupby('user_id')[['price']].agg('sum').reset_index()
#Если пользователь не совершал покупок в этот период, сумма будет равна нулю
df_merged = pd.merge(df_users, df_metrics, on='user_id', how='left').fillna(0)

In [5]:
alpha = 0.05
sample_size = 1000
effect = 0.1

In [6]:
p_values = {'one':[], 'two': [], 'three': []}
values = df_merged['price'].values
mean_ = values.mean()

In [7]:
for _ in tqdm(range(30000)):
    # выбираем случайные группы
    a, b = np.random.choice(values, (2, sample_size), False)
    # добавляем эффект тремя способами
    b_one = b + mean_*effect
    b_two = b * (1 + effect)
    indexes = np.random.choice(np.arange(sample_size), int(sample_size*0.025), False)
    add_value = mean_*effect*sample_size/len(indexes)
    mask = np.zeros(sample_size)
    mask[indexes] += 1
    b_three = b + mask * add_value
    # считаем и сохраняем p-value
    for b_, key in ((b_one, 'one',), (b_two, 'two',), (b_three, 'three',)):
        p_values[key].append(stats.ttest_ind(a, b_).pvalue)

  0%|          | 0/30000 [00:00<?, ?it/s]

In [20]:
# считаем точечные оценки вероятностей ошибки II рода
for key, v in p_values.items():
    errors = (np.array(v)>alpha).astype(int)
    part_errors = np.mean(errors)
    print(f'{key}: part errors = {part_errors:0.4f}')

one: part errors = 0.8172
two: part errors = 0.8317
three: part errors = 0.8284


In [28]:
# проверим, что отличия статистически значимые
print(stats.ttest_ind(p_values['one'], p_values['three']).pvalue)
print(stats.ttest_ind(p_values['two'], p_values['three']).pvalue)

0.006992122753451145
0.01959089490320967
